# Convert old blog posts into JSON

Convert my old blog posts, written in reST and published using Pelican, into a JSON format suitable for importing with `manage.py import_blog_json`. See [Simon's writeup](https://simonwillison.net/2017/Nov/4/import-refs/) for info on the technique.

In [1]:
import pathlib
import json

In [8]:
OLD_ENTRIES_DIR = pathlib.Path('~/c/jacobian.org/content').expanduser()
JSON_OUTPUT = pathlib.Path('~/c/simonwillisonblog/old-blogs.json').expanduser()

In [10]:
for 

Signature: OLD_ENTRIES_DIR.iterdir()
Source:   
    def iterdir(self):
        """Iterate over the files in this directory.  Does not yield any
        result for the special paths '.' and '..'.
        """
        if self._closed:
            self._raise_closed()
        for name in self._accessor.listdir(self):
            if name in {'.', '..'}:
                # Yielding a path object for these makes little sense
                continue
            yield self._make_child_relpath(name)
            if self._closed:
                self._raise_closed()
File:      ~/.pyenv/versions/3.6.4/lib/python3.6/pathlib.py
Type:      method
